<a href="https://colab.research.google.com/github/sanaelk/Springboard/blob/main/Capstone_Two_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os


Objectives

  Develop a model that uses past match results to forecast the outcomes of upcoming women’s international football games.

In [3]:
from google.colab import files



uploaded = files.upload()

Saving goalscorers.csv to goalscorers.csv


In [4]:
#Load the data
df = pd.read_csv('goalscorers.csv')



In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2795 entries, 0 to 2794
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       2795 non-null   object
 1   home_team  2795 non-null   object
 2   away_team  2795 non-null   object
 3   team       2795 non-null   object
 4   scorer     2795 non-null   object
 5   minute     2795 non-null   int64 
 6   own_goal   2795 non-null   bool  
 7   penalty    2795 non-null   bool  
dtypes: bool(2), int64(1), object(5)
memory usage: 136.6+ KB


In [5]:
df.head()


,date,home_team,away_team,team,scorer,minute,own_goal,penalty
0,1984-04-08,England,Denmark,England,Linda Curl,31,False,False
1,1984-04-08,England,Denmark,Denmark,Inge Hindkjær,49,False,True
2,1984-04-08,England,Denmark,England,Elisabeth Deighan,51,False,False
3,1984-04-08,Italy,Sweden,Italy,Carolina Morace,18,False,False
4,1984-04-08,Italy,Sweden,Sweden,Helen Johansson,23,False,False


Need to convert the whole dataset from goal-driven data to match-level data to predict outcomes (Win/Draw/Loss). At the moment each row is whatever goal scored not who won or lost or drew.

In [11]:
df = df.rename(columns={'date': 'datetime'})


In [9]:
#check consistency of names

home_teams = df['home_team'].unique()
away_teams = df['away_team'].unique()

#combine them
all_teams =set(home_teams).union(set(away_teams))

print(all_teams)
print(sorted(all_teams)[:50])

{'Poland', 'Zimbabwe', 'Hungary', 'Argentina', 'Sweden', 'Costa Rica', 'North Macedonia', 'Romania', 'Mexico', 'Haiti', 'Albania', 'Panama', 'Great Britain', 'Slovenia', 'Andorra', 'Canada', 'Russia', 'Latvia', 'Kazakhstan', 'Italy', 'Ghana', 'Malta', 'England', 'Bosnia and Herzegovina', 'Vietnam', 'Netherlands', 'Morocco', 'Colombia', 'Azerbaijan', 'Thailand', 'Serbia', 'New Zealand', 'Armenia', 'Switzerland', 'Northern Ireland', 'Faroe Islands', 'Australia', 'Cameroon', 'Montenegro', 'Greece', 'Brazil', 'France', 'Scotland', 'Denmark', 'Austria', 'Ukraine', 'Georgia', 'Moldova', 'Japan', 'Belgium', 'Slovakia', 'Ivory Coast', 'China PR', 'Kosovo', 'Finland', 'Nigeria', 'Luxembourg', 'Germany', 'South Korea', 'Turkey', 'United States', 'Czech Republic', 'Iceland', 'Cyprus', 'Israel', 'Bulgaria', 'Estonia', 'Wales', 'Jamaica', 'North Korea', 'Taiwan', 'Croatia', 'Zambia', 'Republic of Ireland', 'Lithuania', 'Portugal', 'Ecuador', 'Norway', 'Chile', 'Belarus', 'Equatorial Guinea', 'South

In [15]:
#reorder data according to match win lose or draw

#count goals scored by each team in each match
goals = (df.groupby(['datetime', 'home_team', 'away_team', 'team'])
.size()
.reset_index(name='goals')
)
#create home_goals and away_goals columns
matches = goals.pivot_table(
    index=['datetime', 'home_team', 'away_team'],
    columns='team',
    values='goals',
    fill_value=0,
).reset_index()
matches = (
    df.groupby(['datetime', 'home_team', 'away_team'])
      .apply(lambda x: pd.Series({
          'home_goals': (x['team'] == x['home_team']).sum(),
          'away_goals': (x['team'] == x['away_team']).sum()
      }))
      .reset_index()
)

/tmp/ipython-input-823283507.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


In [18]:
#determine results
def match_result(row):
    if row['home_goals'] > row['away_goals']:
        return 'Home Win'
    elif row['home_goals'] < row['away_goals']:
        return 'Away Win'
    else:
        return 'Draw'
matches.head()

,datetime,home_team,away_team,home_goals,away_goals
0,1984-04-08,England,Denmark,2,1
1,1984-04-08,Italy,Sweden,2,3
2,1984-04-28,Denmark,England,0,1
3,1984-04-28,Sweden,Italy,2,1
4,1984-05-12,Sweden,England,1,0


In [20]:
matches.to_csv('matches.csv', index=False)
from google.colab import files
files.download('matches.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>